<h1>CRTS Light Curve Classification</h1>

<h2>Classifying dataset taking out HPM objects</h2>
<h3>feature_pre_processing.py</h3>
<p><b>remove_hpm:</b> code to remove high proper motion objects from data set</p>

In [ ]:
def remove_hpm(tagged_features):
    tagged_features = tagged_features[tagged_features['tag']!=4]
    return tagged_features

<div  style="margin-top:0px;">
    <div class="pull-left">
<table>
<tr><td>Class</td><td>Precision-with HPM</td><td>Precision without HPM</td></tr>
<tr><td>0-AGN</td><td>0.74</td><td>0.75</td></tr>
<tr ><td>1-Blazar</td><td>0.42</td><td>0.51</td></tr>
<tr><td>2-CV</td><td>0.63</td><td>0.68</td></tr>
<tr><td>3-Flare</td><td>0.53</td><td>0.53</td></tr>
<tr style="color:red;"><td>4-HPM</td><td>0.93</td><td>x</td></tr>
<tr><td>5-SN</td><td>0.72</td><td>0.70</td></tr>
<tr ><td>7-Var</td><td>0.54</td><td>0.60</tr>
<tr style="color:blue;"><td>Mean Score</td><td>0.726</td><td>0.707</tr>
</table>
</div>
<div class="pull-left">
<img style="width:65%;" src="img/score-plot-all-1.png"/> 
</div>

</div>

<h2>Classifying main classes only</h2>
<p>Only cosidered objects originally marked as one of these classes:</p>
<div >
    <div class="pull-left">
<table>
<tr><td>Class</td><td>Precision-main classes only</td><td>Precision- all</td><td>Support-main classes only</td><td>Support-all</td></tr>
<tr style="color:green;"><td>0-AGN</td><td>0.82</td><td>0.74</td><td>740</td><td>980</td></tr>
<tr ><td>1-Blazar</td><td>0.46</td><td>0.42</td><td>101</td><td>116</td></tr>
<tr><td>2-CV</td><td>0.64</td><td>0.63</td><td>335</td><td>390</td></tr>
<tr><td>3-Flare</td><td>0.68</td><td>0.53</td><td>68</td><td>74</td></tr>
<tr><td>5-SN</td><td>0.73</td><td>0.72</td><td>718</td><td>1054</td></tr>
<tr ><td>7-Var</td><td>0.67</td><td>0.54</td><td>47</td><td>60</td></tr>
<tr style="color:blue;"><td>AVG/Total</td><td>0.73</td><td>0.71</td><td>2009</td><td>2881</td></tr>
</table>
</div>




</div>

<div class="pull-left" >
    <img style="width:75%;" src="img/score-plot-main-2.png"/> 
</div>

<h2>Ranking Features</h2>
<p>Several methods were used to rank features</p>
<ul>
    <li>1.-Select-K-Best</li>
    <li>2.-Generic Univariate Select</li>
    <li>3.-Variance Threshold</li>
    <li>4.-Select Percentile</li>
</ul>

<h3>1.-Select K Best, K = 5</h3>
<p>Selects features according to the k highest scores. These scores are calculated using the following functions:</p>
<h4>f-classif:</h4>
<p>Compute the ANOVA F-value for the provided sample. The formula for one way ANOVA F-test is:</p>
<div class="pull-left"><img  style="width:85%;" src="img/anovaftest1.png"/> </div>



<p>Where between group variability is:</p>
<div class="pull-left"><img style="width:85%;" src="img/anovaftest2.png"/> </div>

<div class="pull-left">
<table>
<tr><td><img style="width:65%;" src="img/ni.png"/></td><td>number of observations in the ith group</td></tr>
<tr ><td><img style="width:65%;" src="img/yi.png"/></td><td>sample mean in ith group</td></tr>
<tr ><td><img style="width:55%;" src="img/y.png"/></td><td>overall mean</td></tr>
<tr><td><img style="width:55%;" src="img/k.png"/></td><td>number of groups</td></tr>
</table>
</div>



<p>and within group variability is:</p>
<div class="pull-left"><img style="width:85%;" src="img/anovaftest3.png"/> </div>
<div class="pull-left">
<table>
<tr><td><img style="width:65%;" src="img/yij.png"/></td><td>jth observation in the ith group</td></tr>
<tr ><td><img style="width:65%;" src="img/yi.png"/></td><td>sample mean in ith group</td></tr>
<tr ><td><img style="width:55%;" src="img/n.png"/></td><td>overall sample size</td></tr>
<tr><td><img style="width:55%;" src="img/k.png"/></td><td>number of groups</td></tr>
</table>
</div>


<h4>multi-info-classif:</h4>
<p>Mutual information of two random variables  is a measure of the mutual dependence. More specifically, it quantifies the "amount of information" obtained about one random variable, through the other random variable: </p>
<div class="pull-left"><img  style="width:85%;" src="img/mutualinfo.png"/> </div>

<h3>2.-Generic Univariate Select</h3>
<p>Univariate feature selector. The same 2 functions from above where used</p>
<h3>3.-Variance Threshold 80%</h3>
<p> It removes all features whose variance doesn’t meet some threshold.By default, it removes all zero-variance features, i.e. features that have the same value in all samples.</p>
<div class="pull-left"><img  style="width:80%;" src="img/var.png"/> </div>


<h3>4.-Select Percentile 25</h3>
<p> Removes all but highest scoring percentage of features. It was also computed using f-classif and multi-info-classif</p>

<h3>feature_ranking.py</h3>
<h5>Code to iterate over different feature selection methods and get the lists of most useful ones</h5>

In [ ]:
feature_selection_methods = [
    VarianceThreshold(threshold=(.8 * (1 - .8))),
    SelectKBest(k=5),
    SelectKBest(score_func=mutual_info_classif, k=5),
    SelectPercentile(percentile = 25),
    GenericUnivariateSelect(),
    GenericUnivariateSelect(score_func=mutual_info_classif),
] 

feature_selection_names = [
    "VarianceThreshold 80%",
    "SelectKBest(k=5)",
    "SelectKBest(mutual_info_classif, k=5)",
    "SelectPercentile 25",
    "GenericUnivariateSelect",
    "GenericUnivariateSelect(mutual_info_classif)",
]  

if __name__ == "__main__":
    inputFile = "data/clean_tagged_features.csv"
    data = pd.read_csv(inputFile, sep=",")
    X = data[clean_feature_list]
    Y = data["tag"]
    for i,selection in enumerate(feature_selection_methods):
        fit = selection.fit(X,Y)
        features = selection.get_support(indices=True)
        feature_names = [clean_feature_list[item] for item in features]        
        print(feature_selection_names[i]+": "+ str(feature_names))

<h3>Resulting features</h3>
<div class="pull-left">
<ul>
<li>Amplitude</li>
<li>Small Kurtosis</li>
<li>Standard Deviation</li>
<li>Q31</li>
<li>GSkew</li>
<li>Mean Variance</li>
<li>Percent Difference Flux Percentile</li>
<li>Skew</li>
<li>Autocorrelation Length</li>
<li>Mean</li>
</ul></div>
<div class="pull-left">
<table class="pull-left">
<tr><td>Method</td><td>Features</td></tr>
<tr><td>VarianceThreshold 80%</td><td>[
    'amplitude', 'mean', 'std', 'q31', 'gskew', 'smallKurtosis', 'autocor_length', 'skew']</td></tr>
<tr ><td>SelectKBest(k=5)</td><td>['amplitude', 'std', 'percentDifferenceFluxPercentile', 'q31', 'meanVariance']</td></tr>
<tr ><td>SelectKBest(mutual_info_classif, k=5)</td><td>['amplitude', 'mean', 'percentDifferenceFluxPercentile', 'gskew', 'meanVariance']</td></tr>
<tr><td>SelectPercentile 25</td><td>['amplitude', 'mean', 'std', 'percentDifferenceFluxPercentile', 'q31', 'meanVariance']</td></tr>
<tr><td>GenericUnivariateSelect</td><td>['meanVariance']</td></tr>
<tr><td>GenericUnivariateSelect(mutual_info_classif)</td><td>['gskew']</td></tr>
</table>
</div>


<h3>Results using ranked features</h3>
<div class="pull-left">
<table class="pull-left">
<tr><td>Method</td><td>Mean Scores all-classes</td><td>Mean Scores main-classes</td></tr>
<tr><td>VarianceThreshold 80%</td><td> 0.699</td><td>0.717</td></tr>
<tr ><td>SelectKBest(k=5)</td><td>0.612</td><td>0.648</td></tr>
<tr ><td>SelectKBest(mutual_info_classif, k=5)</td><td> 0.680</td><td>0.696</td></tr>
<tr><td>SelectPercentile 25</td><td>0.661</td><td>0.681</td></tr>
<tr style="color:green;"><td>All Ranked Features</td><td>0.686</td><td>0.724</td></tr>
<tr style="color:blue;"><td>All Features</td><td>0.707</td><td>0.722</td></tr>
</table>
</div>

<h2>Visual Analysis</h2>

<div class="pull-left"><img src="img/fp3.png"/> </div>

<h2>PCA</h2>
<h3>pca.py</h3>
<h5>Code to apply Principal Component Analysis to features and reduce them to 5, 10 and 15</h5>

In [ ]:
#load data
data_dir = dirname(dirname(abspath(__file__)))+"/data/"
output = data_dir+"pca/"
inputFile = data_dir+"clean_tagged_features.csv"
data = pd.read_csv(inputFile, sep=",")
X = data[clean_feature_list]
Y = data["tag"]

#PCA for 3 cases
for n in [5,10,15]:
    pca = PCA(n_components=n)
    new_X = pca.fit_transform(X)
    newdf = pd.DataFrame(new_X)
    newdf["tag"]=Y
    newdf.to_csv(output+str(n)+".csv", sep=',')


<h3>Results using PCA</h3>
<div class="pull-left">
<table class="pull-left">
<tr><td>Method</td><td>Mean Scores all-classes</td><td>Mean Scores main-classes</td></tr>
<tr><td>PCA5</td><td> 0.679</td><td>0.706</td></tr>
<tr><td>PCA10</td><td>0.706</td><td>0.726</td></tr>
<tr><td>PCA15</td><td>0.711</td><td>0.734</td></tr>
</table>
</div>
<div class="pull-left"><img style="width:75%;" src="img/scpca15.png"/> </div>

<h2>Which features should I add next?</h2>
<div class="pull-left"><img src="img/fnext.png"/> </div>

<ul>
    <li>PeriodLS</li>
    <li>StetsonJ</li>
    <li>StetsonK</li>
    <li>Von-Neumann Ratio</li>
    <li>Inter Quartile Range</li>
    <li>Kurtosis</li>
    <li>Robust Median Statistic</li>
</ul>

<h2>Differences in mangintude and time</h2>
<h3>dmdtmapping.py</h3>
<h5>Code to get the dm dt mapping for a light curve</h5>

<p>For each pair of points in a light curve:</p>
<ul>
 <li>calculate difference in magnitude</li>
 <li>calculate difference in time</li>
 <li>End up with p points for a light curve of length n:
    p = n ∗ (n − 1)/2 </li>
 <li>dm and dt are binned and a 2D representatio is obtained</li>
 <li>The 2D representation is then normalized to fit values between 0 and 255</li>
</ul>
 


<p>ranges for dm = ±[0,0.1,0.2,0.3,0.5,1,1.5,2,2.5,3,5,8]</p> 
<p>ranges for dt = [1/145,2/145,3/145,4/145,1/25,2/25,3/25,1.5,2.5,3.5,
4.5,5.5,7,10,20,30,60,90,120,240,600,960,2000,4000]</p>


In [ ]:
featuresdf = pd.DataFrame(columns = fnames)
errorsdf = pd.DataFrame(columns=['filename','error'])

def normalized_image_intensity(nbin,p):
    i = int(225*nbin/p + 0.99999)
    return i

def get_dmdts(lc, tag, objid):
    global featuresdf
    #n = number of points in the light curve
    n = lc.shape[1]
    #p = number of dm and dt for each pair of points
    p = (n*(n-1))/2
    ms = lc[0]
    ts = lc[1]
    #calculate dm
    dms = [abs(y - x) for x, y in it.combinations(ms, 2)]
    binsdm = np.histogram(dms, bins=dmranges)
    #calculate dt
    dts = [abs(y - x) for x, y in it.combinations(ts, 2)]
    binsdt = np.histogram(dts, bins=dtranges)
    #normalize image intensity so it's at most 225
    dm_norm =[normalized_image_intensity(bin, p) for bin in binsdm[0]]
    dt_norm =[normalized_image_intensity(bin, p) for bin in binsdt[0]]
    content = [objid]+dm_norm+dt_norm+[tag]
    feature_row = pd.DataFrame([content], columns = fnames)
    featuresdf = pd.concat([featuresdf,feature_row], ignore_index=True)

<h2>Results so far</h2>

<ul>
 <li>NN and Random Forest are the classifiers that overall are performing best so far</li>
 <li>Features that work best so far: PCA15, ranked and all</li>
 <li>dm and dt did not perform well, but there are 2 things I haven't tried yet
    <ul>
     <li>Changing the ranges of the bins to account for big changes in manitude in small periods of time</li>
     <li>Used dm dts to train a classifier that works well with 2D images, such as CNNs</li>
    </ul>
  </li>
</ul>

<div class="pull-left"><img src="img/compare.png"/> </div>

<h2>Ideas</h2>
<ul>
    <li>Construct a bigger dataset</li>
    <li>Change bin ranges for and dm dt</li>
    <li>Code missing features</li>
    <li>Try different classifiers (like CNN)</li>
    <li>Clustering</li>

</ul>
